# 0. Install and Import Dependencies

In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

  Using cached tensorflow-2.4.1-cp38-cp38-macosx_10_11_x86_64.whl (173.9 MB)
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.4.1 (from versions: none)
ERROR: No matching distribution found for tensorflow-gpu==2.4.1


In [11]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
from multiprocessing import Process
import time, sched
from threading import Thread

In [12]:
CURRENT_LEVEL = "LEVEL_1"
s = sched.scheduler(time.time, time.sleep)
def do_something(): 
    print("Doing stuff...")
    CURRENT_LEVEL = "LEVEL_2"

s.enter(20, 1, do_something)
s.run()

Doing stuff...


# 1. Load Model

In [13]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

# 2. Initialize constants

In [14]:
cap = cv2.VideoCapture(0)

WINDOW_WIDTH = int(cap.get(3))
WINDOW_HEIGHT = int(cap.get(4))
LINE_THICKNESS = 2
color = (0, 255, 255)

In [15]:
left_x1, left_y1 = int(WINDOW_WIDTH / 3), 0
left_x2, left_y2 = int(WINDOW_WIDTH / 3), WINDOW_HEIGHT

right_x1, right_y1 = int((WINDOW_WIDTH / 3) * 2), 0
right_x2, right_y2 = int((WINDOW_WIDTH / 3) * 2), WINDOW_HEIGHT

left_line = (left_x1, left_y1)
right_line = (left_x2, left_y2)

level1 = Level([left_line, right_line])

NameError: name 'Level' is not defined

# 3. Draw levels

In [ ]:
#bl = Botttom left, tr = Top right, p = point
def withinBoundaries(bl, tr, p) :
    print(p[0] > bl[0], p[0] < tr[0], p[1] < bl[1], p[1] > tr[1])
    if (p[0] > bl[0] and p[0] < tr[0] and p[1] < bl[1] and p[1] > tr[1]) :
        return True
    else:
        return False

In [ ]:
def drawLevelOne(frame, joint_points):
    left_x1, left_y1 = int(WINDOW_WIDTH / 3), 0
    left_x2, left_y2 = int(WINDOW_WIDTH / 3), WINDOW_HEIGHT
    bl = [left_x2, left_y2]
    
    right_x1, right_y1 = int((WINDOW_WIDTH / 3) * 2), 0
    right_x2, right_y2 = int((WINDOW_WIDTH / 3) * 2), WINDOW_HEIGHT
    tr = [right_x1, right_y1]
    
    test = (left_x1, left_y1)
    test2 = (left_x2, left_y2)

    for point in joint_points:
        if(withinBoundaries(bl, tr, point)):
            # Green color in BGR
            color = (0, 255, 0)
        else:
            # Red color in BGR
            color = (0, 0, 255)
            break
    
    cv2.line(frame, test, test2, color, thickness=LINE_THICKNESS)
    cv2.line(frame, (right_x1, right_y1), (right_x2, right_y2), color, thickness=LINE_THICKNESS)

In [ ]:
def drawLevelTwo(frame, joint_points):
    left_x1, left_y1 = 0, int(WINDOW_HEIGHT / 3)
    left_x2, left_y2 = WINDOW_WIDTH, int(WINDOW_HEIGHT / 3)
    bl = [left_x2, left_y2]
    
    right_x1, right_y1 = 0, int((WINDOW_HEIGHT / 3) * 2)
    right_x2, right_y2 = WINDOW_WIDTH, int((WINDOW_HEIGHT / 3) * 2)
    tr = [right_x1, right_y1]
    
    test = (left_x1, left_y1)
    test2 = (left_x2, left_y2)

    for point in joint_points:
        if(withinBoundaries(bl, tr, point)):
            # Green color in BGR
            color = (0, 255, 0)
        else:
            # Red color in BGR
            color = (0, 0, 255)
            break
    
    cv2.line(frame, test, test2, color, thickness=LINE_THICKNESS)
    cv2.line(frame, (right_x1, right_y1), (right_x2, right_y2), color, thickness=LINE_THICKNESS)

In [ ]:
def checkPositions(keypoints_with_scores, frame):    
    joint_points = []
    
    for i in range(17):
        joint = np.array(keypoints_with_scores[0][0][i][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
        joint_points.append([joint[1], joint[0]])
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    nose = np.array(keypoints_with_scores[0][0][0][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    coordinates = "(" + str(nose[1]) + ", " + str(nose[0]) + ")"
    cv2.putText(frame, coordinates, (50, 50), font, 1, (255, 0, 0), 2, cv2.LINE_4)
        
    
    if(CURRENT_LEVEL == "LEVEL_1"):
        drawLevelOne(frame, joint_points)
    else:
        drawLevelTwo(frame, joint_points)
        


In [ ]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

# 2. Make Detections

In [21]:
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    
    right_eye = keypoints_with_scores[0][0][1]
    
    # Rendering 
    #draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    checkPositions(keypoints_with_scores, frame)
    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

2022-04-22 12:32:31.116011: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-22 12:32:31.116388: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


True True True True
True True True True
False True True True
True True True True
True True True True
False True True True
True True True True
True True True True
False True True True
True True True True
True True True True
False True True True
True True True True
True True True True
False True True True
True True True True
True True True True
False True True True
True True True True
True True True True
False True True True
True True True True
True True True True
False True True True
True True True True
True True True True
False True True True
True True True True
True True True True
False True True True
True True True True
True True True True
False True True True
True True True True
True True True True
False True True True
True True True True
True True True True
True True True True
True True True True
False True True True
True True True True
True True True True
True True True True
True True True True
False True True True
True True True True
True True True True
False True True True
True 

True True True True
True True True True
True True True True
True True True True
False True True True
True True True True
True True True True
True True True True
True True True True
False True True True
True True True True
True True True True
True True True True
True True True True
False True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True Tr

True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True Tr

False True True True
True True True True
True True True True
False True True True
True True True True
True True True True
False True True True
True True True True
True True True True
False True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
False True True True
True True 

...

True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False True True
True False Tr

True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True

True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True

True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True True True True
True False True True
True True True

AttributeError: 'NoneType' object has no attribute 'copy'

NameError: name 'self' is not defined

In [ ]:
class Level:
    def __init__(self, boundary_pairs):
        self.color = (0, 255, 255)
        self.line_thickness = 2
        self.boundary_pairs = boundary_pairs
    
    
    def checkConstraints(self, keypoints_with_scores):
        print("checksConstraints called. Should change color")
        
    
    def drawLevel(self, frame):
        for boundary_pair in boundary_pairs:
            cv2.line(frame, boundary_pair[0], boundary_pair[1], self.color, thickness=self.line_thickness)
            
            

In [ ]:
    nose = np.array(keypoints_with_scores[0][0][0][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    left_eye = np.array(keypoints_with_scores[0][0][1][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    right_eye = np.array(keypoints_with_scores[0][0][2][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    left_ear = np.array(keypoints_with_scores[0][0][3][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    right_ear = np.array(keypoints_with_scores[0][0][4][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    left_shoulder = np.array(keypoints_with_scores[0][0][5][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    right_shoulder = np.array(keypoints_with_scores[0][0][6][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    left_elbow = np.array(keypoints_with_scores[0][0][7][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    right_elbow = np.array(keypoints_with_scores[0][0][8][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    left_wrist = np.array(keypoints_with_scores[0][0][9][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    right_wrist = np.array(keypoints_with_scores[0][0][10][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    left_hip = np.array(keypoints_with_scores[0][0][11][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    right_hip = np.array(keypoints_with_scores[0][0][12][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    left_knee = np.array(keypoints_with_scores[0][0][13][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    right_knee = np.array(keypoints_with_scores[0][0][14][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    left_ankle = np.array(keypoints_with_scores[0][0][15][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    right_ankle = np.array(keypoints_with_scores[0][0][16][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)